# Convergence de PnP-ISTA avec débruiteurs MMSE

KHOUAJA Ahmed — M2 MIASHS, Lyon 2  
Cours Problèmes Inverses (N. Pustelnik)  
Article : Xu, Sun, Liu, Wohlberg, Kamilov — *IEEE SPL*, 2020

---

## 1. Introduction

En traitement du signal, on cherche à reconstruire un signal $\mathbf{x}$ à partir de mesures dégradées $\mathbf{y} = H\mathbf{x} + \mathbf{e}$, où $H$ modélise l'acquisition et $\mathbf{e}$ le bruit. Ce problème est mal posé et on régularise pour obtenir une solution — c'est le sujet des premiers chapitres du cours.

L'approche Plug-and-Play (Venkatakrishnan et al. 2013) remplace le régulariseur par un débruiteur dans les itérations d'un algorithme proximal. L'article étudié ici prouve la convergence de PnP-ISTA quand le débruiteur est un estimateur MMSE. Jusqu'ici, les preuves de convergence supposaient la nonexpansivité du débruiteur, ce qui n'est pas le cas du MMSE. Xu et al. contournent le problème via une approche de majorisation-minimisation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

np.random.seed(2024)

## 2. Cadre théorique : ISTA et Plug-and-Play

La reconstruction régularisée revient à résoudre $\hat{\mathbf{x}} = \arg\min_{\mathbf{x}} g(\mathbf{x}) + h(\mathbf{x})$ avec $g(\mathbf{x}) = \frac{1}{2}\|\mathbf{y} - H\mathbf{x}\|^2$ et $h$ un régulariseur ($\ell_1$, TV, etc.).

L'algorithme ISTA, ou Forward-Backward (chap. 4 du cours), itère :
$$z_t = x_{t-1} - \gamma \nabla g(x_{t-1}), \quad x_t = \mathrm{prox}_{\gamma h}(z_t)$$
avec $\gamma \leq 1/L$, $L$ constante de Lipschitz de $\nabla g$. L'opérateur proximal admet une interprétation bayésienne : si $h = -\log p_x$, le proximal correspond au débruiteur MAP pour un bruit gaussien de variance $\gamma$. L'approche PnP remplace ce proximal par un débruiteur $D_\sigma$ :

$$z_t = x_{t-1} - \gamma \nabla g(x_{t-1}), \qquad x_t = D_\sigma(z_t)$$

Le débruiteur MAP est nonexpansif et les preuves de convergence existantes reposent sur cette propriété (Chan 2017, Ryu 2019). Le débruiteur MMSE ($\mathbb{E}[\mathbf{x}|\mathbf{z}]$), qui minimise l'erreur quadratique, est en général expansif. Or la majorité des débruiteurs utilisés en pratique — DnCNN entraîné en MSE, BM3D — sont des approximations du MMSE. La formule de Tweedie $D_\sigma(\mathbf{z}) = \mathbf{z} + \sigma^2 \nabla \log p_z(\mathbf{z})$ permet de montrer que le MMSE est le proximal d'un $h_\mathrm{mmse}$ possiblement non convexe (Gribonval 2011).

## 3. Résultat de convergence

Sous trois hypothèses — prior non-dégénéré, gradient $L$-Lipschitz, $f = g+h_\mathrm{mmse}$ bornée inférieurement — la suite $\{f(x_t)\}$ produite par PnP-ISTA avec le MMSE décroît de façon monotone et $\|\nabla f(x_t)\| \to 0$. La preuve passe par un majorant quadratique :
$$\bar\mu(x,s) = g(s) + \langle \nabla g(s), x-s\rangle + \frac{1}{2\gamma}\|x-s\|^2 + h_\mathrm{mmse}(x)$$
Comme $D_\sigma = \mathrm{prox}_{\gamma h_\mathrm{mmse}}$, chaque itération minimise exactement ce majorant. La suite, décroissante et bornée, converge. Pas besoin de convexité ni de nonexpansivité.

## 4. Résultats de l'article

Les auteurs testent sur du compressive sensing : signal Bernoulli-Gaussien ($n=4096$, $\alpha = 0.1$), matrice gaussienne i.i.d., 20 dB de SNR.

La convergence monotone de $f(x_t)/f(x_0)$ est vérifiée. Le DnCNN entraîné en MSE reproduit le MMSE analytique, confirmant que ces réseaux sont bien des approximations MMSE. PnP-ISTA reste en dessous de GAMP, ce qui s'explique par le fait qu'ISTA suppose un bruit AWGN dans les résidus, hypothèse fausse en pratique.

## 5. Reproduction

Deux expériences : compressive sensing 1D avec le MMSE analytique, puis défloutage 2D avec DnCNN via DeepInverse. On travaille en dimension réduite par rapport à l'article.

### 5.1 Compressive sensing avec MMSE exact

Pour un prior Bernoulli-Gaussien i.i.d., le débruiteur MMSE s'exprime analytiquement composante par composante : $D_\sigma(z_i) = w(z_i) \frac{\sigma_x^2}{\sigma^2 + \sigma_x^2} z_i$ avec $w(z_i) = P(x_i \neq 0 \mid z_i)$.